## Import Modules

In [121]:
# import modules
import pandas as pd
import numpy as np

## Loading Data

In [122]:
# read data
training_data = pd.read_csv('data/players_21.csv')
testing_data = pd.read_csv('data/players_22.csv')

/tmp/ipykernel_6850/653244438.py:3: DtypeWarning: Columns (25,108) have mixed types. Specify dtype option on import or set low_memory=False.
  testing_data = pd.read_csv('data/players_22.csv')


## Data Preprocessing

### Remove columns with na values that exceed 30%

In [123]:
# Calculate the percentage of na values in each column
na_percentages = training_data.isna().sum() / len(training_data) * 100

# Select the columns where the percentage of na values exceeds 30%
cols_to_drop = na_percentages[na_percentages > 30].index

# Drop the selected columns
training_data.drop(cols_to_drop, axis=1, inplace=True)
testing_data.drop(cols_to_drop, axis=1, inplace=True)

### Remove columns where the values do not obviously contribute a player's overall rating

In [124]:
# urls do not help to predict a player's rating
# remove columns that have 'url' in their name
cols_to_drop = [col for col in training_data.columns if 'url' in col]
training_data.drop(cols_to_drop, axis=1, inplace=True)
testing_data.drop(cols_to_drop, axis=1, inplace=True)

In [125]:
# columns that obviously do not contribute to a player's rating
cols_to_drop = [
    "age",
    "sofifa_id",
    "short_name",
    "long_name",
    "real_face",
]

# drop the columns
training_data.drop(cols_to_drop, axis=1, inplace=True)
testing_data.drop(cols_to_drop, axis=1, inplace=True)

In [126]:
# columns where input would not be available at the time of prediction
cols_to_drop = [
    "gk", "rb", "rcb", "cb", "lcb", "lb", "rwb", "rdm", "cdm", "ldm", "lwb", "rm", "rcm", "cm", "lcm", "lm", "ram", "cam", "lam", "rw", "rf", "cf", "lf", "lw", "rs", "st", "ls", "club_joined", "club_contract_valid_until"
]

# drop the columns
training_data.drop(cols_to_drop, axis=1, inplace=True)
testing_data.drop(cols_to_drop, axis=1, inplace=True)

In [127]:
# remove club, national and league info. They do not explicitly determine a player's rating
cols_to_drop = [
    "club_name", "league_name", "league_level", "club_jersey_number", "nationality_id", "nationality_name", "value_eur", "release_clause_eur", "club_team_id"
]

# drop the columns
training_data.drop(cols_to_drop, axis=1, inplace=True)
testing_data.drop(cols_to_drop, axis=1, inplace=True)

### Setup training and testing data

In [128]:
trainX = training_data.drop('overall', axis=1)
trainY = training_data['overall']
testX = testing_data.drop('overall', axis=1)
testY = testing_data['overall']

#### Imputing data

In [137]:
trainX.shape[1]

53

In [134]:
# show all missing fields in the columns
# trainX.isna().sum()

player_positions                  0
potential                         0
wage_eur                        225
dob                               0
height_cm                         0
weight_kg                         0
club_position                   225
preferred_foot                    0
weak_foot                         0
skill_moves                       0
international_reputation          0
work_rate                         0
body_type                         0
pace                           2083
shooting                       2083
passing                        2083
dribbling                      2083
defending                      2083
physic                         2083
attacking_crossing                0
attacking_finishing               0
attacking_heading_accuracy        0
attacking_short_passing           0
attacking_volleys                 0
skill_dribbling                   0
skill_curve                       0
skill_fk_accuracy                 0
skill_long_passing          

#### Encoding data

In [130]:
# trainX.info()

In [131]:
# # use pd.factorize to convert categorical columns to numerical
# # check if dtype is object

# # get categorical columns
# cat_cols = [col for col in trainX.columns if trainX[col].dtype == 'object']

# # factorize the categorical columns
# for col in cat_cols:
#     trainX[col], c1 = pd.factorize(trainX[col])
#     testX[col], c2 = pd.factorize(testX[col])

In [132]:
# trainX.info()